In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.8e}'.format

import configparser
from collections import defaultdict
from src.character import Character, CharacterType 
from src.stats import random_stats
from src.selection import stochastic_tournament_selection, deterministic_tournament_selection, add_relative_accumulate, elitist_selection, roulette_wheel_selection, universal_selection, boltzmann_selection, rank_based_selection
from src.genes import  encode_genes, decode_genes
from src.mutation import one_gene_mutation, multi_gene_mutation, multi_gene_mutation_uniform

In [2]:
import sys
sys.path.append("src")

from src.population import generate_init_population, eval_performace
from src.crossover import single_point_crossover, two_point_crossover, uniform_crossover, annular_crossover, normalize_chromosome


In [3]:
def read_config(filename):
    config = configparser.ConfigParser()
    config.read(filename)
    params = defaultdict(dict)
    for section in config.sections():
        for key, value in config.items(section):
            params[section][key] = value
    return params

params = read_config('config_file.config')
params

defaultdict(dict,
            {'CHARACTER': {'type': 'CharacterType.ARCHER'},
             'N': {'n': '200'},
             'M': {'m': '150'},
             'K': {'k': '120'},
             'GENERATIONS': {'number': '500'},
             'METHOD1PERCENTAGE': {'percentage': '0.4'},
             'SELECTION_METHOD': {'method1': "'elitist'",
              'method2': "'roulette_wheel'"},
             'BOLTZMANN': {'t_0': '50',
              't_offset': '10',
              'delta_t': '0.015',
              'm': '0.01'},
             'deterministic_tournament_selection': {'tournament_size': '50'},
             'stochastic_tournament_selection': {'threshold': '0.6'},
             'metodo_cruza': {'crossover': "'one_point'"},
             'MUTATION': {'mutation_type': "'multi_gene'",
              'mutation_rate': '0.7'},
             'METHODRePERCENTAGE': {'percentage': '0.6'},
             'metodo_reemplazo': {'metodo1': "'rank_based_selection'",
              'metodo2': "'deterministic_tournamen

In [4]:
from collections import namedtuple

def parameters():
    
    config_params = read_config('config_file.config')

    # Personaje 
    type = eval(config_params['CHARACTER']['type'])
    
    # sum( stats ) = 150
    maxStatsValue = eval(config_params['M']['m'])
    
    # N
    populationNumber = eval(config_params['N']['n'])
    # K
    k = eval(config_params['K']['k'])

    # Numero máximo de generaciones - criterio de corte
    numberOfGenerations = eval(config_params['GENERATIONS']['number'])
    
    # A%
    method1Percentage = eval(config_params['METHOD1PERCENTAGE']['percentage'])
    selectionMethod1 = eval(config_params['SELECTION_METHOD']['method1'])
    selectionMethod2 = eval(config_params['SELECTION_METHOD']['method2'])
   
    # CrossOver
    metodo_cruza = eval(config_params['metodo_cruza']['crossover'])
 
    # Mutation
    mutation_type = eval(config_params['MUTATION']['mutation_type'])
    mutation_rate = eval(config_params['MUTATION']['mutation_rate'])

    # B%
    methodReplacePercentage = eval(config_params['METHODRePERCENTAGE']['percentage'])
    metodo_reemplazo3 = eval(config_params['metodo_reemplazo']['metodo1'])
    metodo_reemplazo4 = eval(config_params['metodo_reemplazo']['metodo2'])

    cutCriteria = eval(config_params['CUT_CRITERIA']['criteria_type'])
    
    # Selector parameters
    # M
    tournament_size = eval(config_params['deterministic_tournament_selection']['tournament_size'])
    # stochastic_tournament_selection
    threhold = eval(config_params['stochastic_tournament_selection']['threshold'])
    # Boltzmann parameters
    boltzmannT_0 = eval(config_params['BOLTZMANN']['t_0'])
    boltzmannT_c = eval(config_params['BOLTZMANN']['t_offset'])
    boltzmannM = eval(config_params['BOLTZMANN']['m'])

    Param = namedtuple('Param',[
        'character',
        'maxStatsValue',
        'populationNumber',
        'k',
        'numberOfGenerations',
        'method1Percentage',
        'selectionMethod1',
        'selectionMethod2',
        'metodo_cruza',
        'mutation_type',
        'mutation_rate',
        'methodReplacePercentage',
        'metodo_reemplazo3',
        'metodo_reemplazo4',
        'cutCriteria',
        'tournament_size',
        'threhold',
        'boltzmannT_0',
        'boltzmannT_c',
        'boltzmannM'
    ])    

    parametros = Param(
                character=type,
                maxStatsValue=maxStatsValue,
                populationNumber=populationNumber,
                k=k,
                numberOfGenerations=numberOfGenerations,
                method1Percentage=method1Percentage,
                selectionMethod1=selectionMethod1,
                selectionMethod2=selectionMethod2,
                metodo_cruza=metodo_cruza,
                mutation_type=mutation_type,
                mutation_rate=mutation_rate,
                methodReplacePercentage=methodReplacePercentage,
                metodo_reemplazo3=metodo_reemplazo3,
                metodo_reemplazo4=metodo_reemplazo4,
                cutCriteria=cutCriteria,
                tournament_size=tournament_size,
                threhold=threhold,
                boltzmannT_0=boltzmannT_0,
                boltzmannT_c=boltzmannT_c,
                boltzmannM=boltzmannM
    )

    return parametros

p = parameters()


In [5]:
from src.character import Character, CharacterType 
import sys
sys.path.append("src")

from src.population import generate_init_population

#GENERACION 0

generation_0 = generate_init_population(p.populationNumber,p.maxStatsValue,p.character)
generacion = 0


In [6]:
# GUARDO EL MAS APTO DE LA GENERACION 0
# Escribe la poblacion inicial.
generation_0.insert(0, 'generation', int(generacion))
mas_apto_G0 = generation_0.head(1)
mas_apto_G0.to_csv(f'datos-Prueba1.csv', mode='w', index=False)


In [7]:

def method_selector(Kt, method1Percentage): 
    # Selecciono K individuos
    # Utilizo el porcentaje para cada método
    K1 = int(np.round(Kt * method1Percentage))
    K2 = Kt - K1
    return K1, K2

k1, k2 = method_selector(p.k, p.method1Percentage)

In [8]:
current_generation = generation_0
generacion = 0

In [9]:
def selection_method(poblacion, ki, params, method, generation):
    
    print(f' Generacion: {generation}')
    poblacion_rel_acu = add_relative_accumulate(poblacion)

    if(method == 'elitist'):
        selected = elitist_selection(poblacion_rel_acu, ki)
    elif(method == 'roulette_wheel'):
        selected = roulette_wheel_selection(poblacion_rel_acu, ki)
    elif(method == 'boltzmann'):
        selected = boltzmann_selection(poblacion_rel_acu, ki, generation, params.boltzmannT_0,params.boltzmannT_c,params.boltzmannM)
        #print(newPopulationRoulette)
    elif(method == 'universal_selection'):
        selected = universal_selection(poblacion_rel_acu, ki)
        #print(newPopulationRoulette)
    elif(method == 'deterministic_tournament'):
        selected = deterministic_tournament_selection(poblacion_rel_acu, params.tournament_size, ki)
        #print(newPopulationRoulette)
    elif(method == 'stochastic_tournament'):
        selected = stochastic_tournament_selection(poblacion_rel_acu, ki, params.threshold)
        #print(newPopulationRoulette)
    elif(method == 'rank_based'):
        selected = rank_based_selection(poblacion_rel_acu, ki)
        #print(newPopulationRoulette)
    
    return selected

# SELECCION DE PADRES
generacion_sel1 = selection_method(current_generation, k1, p, p.selectionMethod1, generacion)
generacion_sel2 = selection_method(current_generation, k2, p, p.selectionMethod2, generacion)

# SELECCION TOTAL: K padres
selection_total = pd.concat([generacion_sel1, generacion_sel2])

selection_total.drop(columns=['performance_relative'], inplace=True)
selection_total.drop(columns=['performance_accumulated'], inplace=True)
selection_total = selection_total.sort_values(by='performance', ascending=False)
selection_total.reset_index(drop=True, inplace=True)


 Generacion: 0
 Generacion: 0


In [10]:
cromosomas = encode_genes(selection_total)

cromosomas

,strength,agility,expertise,resistance,life,height
1,01010100,00000100,00110001,00000111,00000110,000
2,01001000,00001001,00101011,00001011,00001111,110
3,00110010,00010011,00111001,00001101,00001011,111
4,00110010,00010011,00111001,00001101,00001011,111
5,01010000,00010000,00100100,00000110,00001100,111
...,...,...,...,...,...,...
116,00000101,00110100,01000000,00000100,00011001,110
117,00001110,00010100,00010000,00101000,00111100,101
118,00011010,00001001,00001000,00011101,01001110,011
119,00000011,00011100,00110011,00110001,00010011,010


In [11]:
def crossover_method(poblacion, method):
    
    if(method == 'one_point'):
        crossed = single_point_crossover(poblacion)
    elif(method == 'two_point'):
        crossed = two_point_crossover(poblacion)
    elif(method == 'uniform'):
        crossed = uniform_crossover(poblacion)
    elif(method == 'annular'):
        crossed = annular_crossover(poblacion)
    
    return crossed

crossed_result = crossover_method(cromosomas, p.metodo_cruza)

crossed_result

,strength,agility,expertise,resistance,life,height
0,01001000,00001001,00101011,00000111,00000111,110
1,01010100,00000100,00110001,00001011,00001110,000
2,00110010,00010011,00111001,00001101,00001011,111
3,00110010,00010011,00111001,00001101,00001011,111
4,01010000,00010000,00100100,00000110,00001100,111
...,...,...,...,...,...,...
115,00000010,00110011,01000111,00011010,00010000,110
116,00011010,00001001,00001000,00111101,00111110,111
117,00001110,00010100,00010000,00001000,01001100,001
118,00000011,00011110,00001010,00110000,00011011,011


In [12]:
def mutation_method(poblacion, rate, method):
    
    if(method == 'one_gene'):
        mutated = one_gene_mutation(poblacion, rate)
    elif(method == 'multi_gene'):
        mutated = multi_gene_mutation(poblacion, rate)
    elif(method == 'multi_gene_mutation_uniform'):
        mutated = multi_gene_mutation_uniform(poblacion, rate)
    
    return mutated

mutation_result = mutation_method(crossed_result, p.mutation_rate, p.mutation_type)

mutation_result

     strength   agility expertise resistance      life height
0    01001000  00001001  00101011   00000111  00000111    110
1    01010100  00000100  00110001   00001011  00001110    000
2    00110010  00010011  00111001   00001101  00001011    111
3    00110010  00010011  00111001   00001101  00001011    111
4    01010000  00010000  00100100   00000110  00001100    111
..        ...       ...       ...        ...       ...    ...
115  00000010  00110011  01000111   00011010  00010000    110
116  00011010  00001001  00001000   00111101  00111110    111
117  00001110  00010100  00010000   00001000  01001100    001
118  00000011  00011110  00001010   00110000  00011011    011
119  00000111  00001000  00110011   01010001  00100011    010

[120 rows x 6 columns]


,strength,agility,expertise,resistance,life,height
0,10101100,10111110,01010100,11111101,00000111,110
1,11110001,10000110,00011110,11110101,00110011,000
2,00110010,01100101,11000100,00001101,01100100,001
3,11001001,01001100,11011111,11100010,00010101,000
4,10000110,01011111,00100100,00000110,10110011,000
...,...,...,...,...,...,...
115,11100101,10001110,01000111,00011010,11101111,110
116,01100011,01111001,00001000,00111101,11001100,110
117,11110011,11001011,00010000,11111111,10110111,110
118,00000011,10100101,00001010,11001111,01100000,100


In [13]:
cromosomas_norm = normalize_chromosome(mutation_result)

cromosomas_norm

,strength,agility,expertise,resistance,life,height
1,00100100,00101000,00010001,00110101,00000001,110
2,00110011,00011100,00000110,00110100,00001010,000
3,00010000,00100000,00111111,00000100,00100000,001
4,00101000,00001111,00101100,00101101,00000100,000
5,00101100,00011111,00001100,00000010,00111011,000
...,...,...,...,...,...,...
116,00110000,00011110,00001111,00000101,00110010,110
117,00011110,00100100,00000010,00010010,00111110,110
118,00101000,00100001,00000010,00101010,00011110,110
119,00000000,00110011,00000011,01000000,00011101,100


In [14]:
cromosomas_norm_decode = decode_genes(cromosomas_norm)
offspring = eval_performace(cromosomas_norm_decode,p.character)

noffspring = offspring.sort_values(by='performance', ascending=False)
noffspring.reset_index(drop=True, inplace=True)

offspring

,strength,agility,expertise,resistance,life,height,characterType,performance
0,6.80000000e+01,1.10000000e+01,6.10000000e+01,1.00000000e+00,6.00000000e+00,1.40000000e+00,CharacterType.ARCHER,1.06742353e+03
1,5.30000000e+01,2.20000000e+01,4.80000000e+01,8.00000000e+00,1.60000000e+01,1.80000000e+00,CharacterType.ARCHER,7.67101659e+02
2,6.00000000e+01,7.00000000e+00,3.90000000e+01,0.00000000e+00,4.20000000e+01,1.90000000e+00,CharacterType.ARCHER,6.95729406e+02
3,7.00000000e+01,2.90000000e+01,3.50000000e+01,6.00000000e+00,7.00000000e+00,1.30000000e+00,CharacterType.ARCHER,6.77492999e+02
4,5.80000000e+01,5.70000000e+01,3.20000000e+01,1.00000000e+00,0.00000000e+00,1.50000000e+00,CharacterType.ARCHER,6.62609977e+02
...,...,...,...,...,...,...,...,...
115,5.00000000e+00,6.10000000e+01,4.00000000e+00,1.30000000e+01,6.40000000e+01,1.40000000e+00,CharacterType.ARCHER,2.31354446e+01
116,6.00000000e+00,6.90000000e+01,0.00000000e+00,5.00000000e+00,6.80000000e+01,1.60000000e+00,CharacterType.ARCHER,1.91734714e+01
117,1.00000000e+00,4.80000000e+01,3.80000000e+01,4.80000000e+01,1.30000000e+01,1.50000000e+00,CharacterType.ARCHER,1.76154988e+01
118,1.00000000e+00,4.00000000e+01,2.40000000e+01,3.70000000e+01,4.40000000e+01,1.50000000e+00,CharacterType.ARCHER,1.73463828e+01


In [ ]:
# REEMPLAZO


In [15]:
generacion += 1
# GUARDO EL MAS APTO DE LA GENERACION NUEVA
# Escribe la poblacion inicial.
"""
new_generation.insert(0, 'generation', int(generacion))
mas_apto_NG = new_generation.head(1)
mas_apto_NG.to_csv(f'datos-Prueba1.csv', mode='a', header=False, index=False)
"""